In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bakloli/downloaded_image.jpg
/kaggle/input/amazon-dataset/dataset/._sample_test_out.csv
/kaggle/input/amazon-dataset/dataset/._train.csv
/kaggle/input/amazon-dataset/dataset/._sample_test.csv
/kaggle/input/amazon-dataset/dataset/._sample_test_out_fail.csv
/kaggle/input/amazon-dataset/dataset/._test.csv
/kaggle/input/amazon-dataset/src/._constants.py
/kaggle/input/amazon-dataset/src/._utils.py
/kaggle/input/amazon-dataset/src/._sanity.py
/kaggle/input/amazon-dataset/src/._test.ipynb
/kaggle/input/amazon-dataset/student_resource 3/._src
/kaggle/input/amazon-dataset/student_resource 3/._sample_code.py
/kaggle/input/amazon-dataset/student_resource 3/README.md
/kaggle/input/amazon-dataset/student_resource 3/._README.md
/kaggle/input/amazon-dataset/student_resource 3/._dataset
/kaggle/input/amazon-dataset/student_resource 3/sample_code.py
/kaggle/input/amazon-dataset/student_resource 3/dataset/sample_test.csv
/kaggle/input/amazon-dataset/student_resource 3/dataset/._sample_test

In [3]:
df = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [4]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [5]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


# For single image extraction

1) To see detailed extraction

In [5]:
"""import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Define regex patterns based on the allowed units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units set based on entity_unit_map
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Preprocessing function to clean up OCR text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Replace "O" with "0" (to handle common OCR errors like "140OmG")
    text = text.replace("o", "0")
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any unnecessary punctuation that might interfere with matching
    text = re.sub(r'[^\w\s.,]', '', text)
    
    return text

# Function to extract various units from the text
def extract_units(text, entity_name):
    extracted_units = set()

    # Preprocess the text before extracting units
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        unit_list = entity_unit_map[entity_name]
        for unit in unit_list:
            if unit in patterns:
                pattern = patterns[unit]
                matches = re.findall(pattern, cleaned_text)
                if matches:
                    # Format as "value unit" and store unique entries
                    for match in matches:
                        value = float(match[0])
                        formatted_entry = f"{value} {unit}"
                        extracted_units.add(formatted_entry)

    return extracted_units

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to process each image URL
def process_image_url(image_url, entity_name):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units from the final text based on the entity_name
    units = extract_units(final_text, entity_name)
    
    return final_text, best_angle, units

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Limit to first 30 links for processing
df = df.head(30)

# Initialize a list to store results
results = []

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_link'
    entity_name = row['entity_name']  # Assuming this column specifies what entity to extract (e.g., weight, volume)
    
    print(f"Processing image from URL: {image_url} for entity: {entity_name}")
    
    final_text, best_angle, units = process_image_url(image_url, entity_name)
    
    # Prepare the result row
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)  # Convert set to list for the predicted result
    }
    
    # Append the result to the list
    results.append(result_row)
    
    # Print results for each image
    print(f"Extracted Text (Best angle {best_angle} degrees): {final_text}")
    print(f"Extracted Units for {entity_name}: {result_row['predicted']}")
    print("--------------------------------------------------")

# Create a new DataFrame from results and save it as a CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)
"""

'import easyocr\nimport pandas as pd\nfrom PIL import Image\nimport numpy as np\nimport requests\nfrom io import BytesIO\nimport re\n\n# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)\nreader = easyocr.Reader([\'en\'], gpu=True)\n\n# Define regex patterns based on the allowed units\npatterns = {\n    \'gram\': r\'(\\d+(\\.\\d+)?)\\s*(g|grams?)\',\n    \'kilogram\': r\'(\\d+(\\.\\d+)?)\\s*(kg|kilograms?)\',\n    \'milligram\': r\'(\\d+(\\.\\d+)?)\\s*(mg|milligrams?)\',\n    \'microgram\': r\'(\\d+(\\.\\d+)?)\\s*(µg|micrograms?)\',\n    \'ounce\': r\'(\\d+(\\.\\d+)?)\\s*(oz|ounces?)\',\n    \'pound\': r\'(\\d+(\\.\\d+)?)\\s*(lb|pounds?)\',\n    \'ton\': r\'(\\d+(\\.\\d+)?)\\s*(ton|tons?)\',\n    \'centimetre\': r\'(\\d+(\\.\\d+)?)\\s*(cm|centimetre|centimeters?)\',\n    \'foot\': r\'(\\d+(\\.\\d+)?)\\s*(ft|foot|feet)\',\n    \'inch\': r\'(\\d+(\\.\\d+)?)\\s*(in|inch|inches)\',\n    \'metre\': r\'(\\d+(\\.\\d+)?)\\s*(m|metre|meters?)\',\n    \'millimetr

2)Didn't want to see details

In [ ]:
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import urllib
import re
import warnings
import os
import multiprocessing
from functools import partial
from tqdm import tqdm
from pathlib import Path
import time

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Regex patterns for units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Download an image from URL with retries and placeholder on failure
def download_image(image_link, save_folder, retries=3, delay=3):
    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    # Skip if image already exists
    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception:
            time.sleep(delay)

    # Create a placeholder image in case of failure
    Image.new('RGB', (100, 100), color='black').save(image_save_path)

# Parallel image downloading function
def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    download_image_partial = partial(download_image, save_folder=download_folder)
    with multiprocessing.Pool(64) as pool:
        list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
        pool.close()
        pool.join()

# Preprocess text (lowercase, handle common OCR errors)
def preprocess_text(text):
    text = text.lower().replace("o", "0")
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,]', '', text)
    return text

# Extract units from text based on entity type
def extract_units(text, entity_name):
    extracted_units = set()
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        for unit in entity_unit_map[entity_name]:
            if unit in patterns:
                matches = re.findall(patterns[unit], cleaned_text)
                for match in matches:
                    value = float(match[0])
                    extracted_units.add(f"{value} {unit}")
    return extracted_units

# Perform OCR and extract best text by rotating image
def rotate_and_extract(image):
    best_text, best_angle = "", 0
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        rotated_img_np = np.array(rotated_img)
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        if len(final_text.strip()) > len(best_text.strip()):
            best_text, best_angle = final_text, angle
    return best_text, best_angle

# Process image URL
def process_image_url(image_url, entity_name, download_folder):
    # Download and open image
    filename = Path(image_url).name
    image_path = os.path.join(download_folder, filename)
    image = Image.open(image_path)
    
    # Extract text using OCR and find best rotation
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units
    units = extract_units(final_text, entity_name)
    return final_text, best_angle, units

# Load CSV file (replace with your path)
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path).head(10000)  # Process first 100 links

# Image folder to save the downloaded images
download_folder = "downloaded_images"

# Download images in parallel
print("Starting image download...")
download_images(df['image_link'], download_folder)

# Initialize list to store results
results = []

# Start tracking time
start_time = time.time()

# Process each image and extract relevant data
for index, row in df.iterrows():
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Process image to get text, angle, and units
    final_text, best_angle, units = process_image_url(image_url, entity_name, download_folder)

    # Add results
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)
    }
    results.append(result_row)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    remaining_images = len(df) - (index + 1)
    estimated_total_time = (elapsed_time / (index + 1)) * len(df)
    remaining_time = estimated_total_time - elapsed_time

    # Print countdown
    print(f"Processed {index + 1}/{len(df)} images. Estimated time remaining: {remaining_time // 60:.0f}m {remaining_time % 60:.0f}s")

# Create DataFrame and save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)

print(f"Processed {len(results)} images.")


Starting image download...


 59%|█████▉    | 5939/10000 [07:02<4:15:47,  3.78s/it]

In [44]:
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import urllib
import re
import warnings
import os
import multiprocessing
from functools import partial
from tqdm import tqdm
from pathlib import Path
import time

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Regex patterns for units (you should define these according to your specific needs)
patterns = {
    'cm': r'(\d+(?:\.\d+)?)\s*cm',
    'mm': r'(\d+(?:\.\d+)?)\s*mm',
    'kg': r'(\d+(?:\.\d+)?)\s*kg',
    'm': r'(\d+(?:\.\d+)?)\s*m',
}

# Map entity names to their corresponding unit patterns (example data)
entity_unit_map = {
    'height': ['cm', 'm'],
    'weight': ['kg'],
    'thickness': ['mm'],
}

# Allowed units (unchanged)
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Optimized image downloading with existing check and placeholder
def download_image(image_link, save_folder, retries=3, delay=3):
    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    # Skip if image already exists
    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception:
            time.sleep(delay)

    # Create a placeholder image in case of failure
    Image.new('RGB', (100, 100), color='black').save(image_save_path)

# Optimized parallel image downloading
def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    download_image_partial = partial(download_image, save_folder=download_folder)
    # Limit the pool size to avoid overloading the system
    pool_size = min(multiprocessing.cpu_count(), len(image_links))
    
    with multiprocessing.Pool(pool_size) as pool:
        list(tqdm(pool.imap_unordered(download_image_partial, image_links), total=len(image_links)))
        pool.close()
        pool.join()

# Preprocess text (unchanged)
def preprocess_text(text):
    text = text.lower().replace("o", "0")
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,]', '', text)
    return text

# Extract units from text
def extract_units(text, entity_name):
    extracted_units = set()
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        for unit in entity_unit_map[entity_name]:
            if unit in patterns:
                matches = re.findall(patterns[unit], cleaned_text)
                for match in matches:
                    value = float(match[0])
                    extracted_units.add(f"{value} {unit}")
    return extracted_units

# Perform OCR and extract best text by rotating image
def rotate_and_extract(image):
    # Convert image to numpy array once
    image_np = np.array(image)
    
    best_text, best_angle = "", 0
    for angle in [0, 90]:
        rotated_img_np = np.rot90(image_np, k=angle // 90)
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        if len(final_text.strip()) > len(best_text.strip()):
            best_text, best_angle = final_text, angle
        # Early stop if text is long enough (adjust threshold as needed)
        if len(best_text.strip()) > 100: 
            break
    return best_text, best_angle

# Process image URL with reduced I/O overhead
def process_image_url(image_url, entity_name, download_folder):
    # Reuse already downloaded image
    filename = Path(image_url).name
    image_path = os.path.join(download_folder, filename)
    image = Image.open(image_path)
    
    # Extract text using OCR and find best rotation
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units
    units = extract_units(final_text, entity_name)
    return final_text, best_angle, units

# Load CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path).head(1000)  # Process first 1000 links

# Image folder to save the downloaded images
download_folder = "downloaded_images"

# Download images in parallel (optimized pool size)
print("Starting image download...")
download_images(df['image_link'], download_folder)

# Initialize list to store results
results = []

# Start tracking time
start_time = time.time()

# Process each image and extract relevant data
for index, row in df.iterrows():
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Process image to get text, angle, and units
    final_text, best_angle, units = process_image_url(image_url, entity_name, download_folder)

    # Add results
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)
    }
    results.append(result_row)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    remaining_images = len(df) - (index + 1)
    estimated_total_time = (elapsed_time / (index + 1)) * len(df)
    remaining_time = estimated_total_time - elapsed_time

    # Print countdown
    print(f"Processed {index + 1}/{len(df)} images. Estimated time remaining: {remaining_time // 60:.0f}m {remaining_time % 60:.0f}s")

# Create DataFrame and save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)

print(f"Processed {len(results)} images.")


Starting image download...



  0%|          | 0/1000 [00:00<?, ?it/s]Process SpawnPoolWorker-874:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/conda/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'download_image' on <module '__main__' (built-in)>
Process SpawnPoolWorker-875:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.10/multiprocessing/pool.py", line 114, in w

KeyboardInterrupt: 

In [7]:
"""
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re
import warnings

# Suppress FutureWarnings and other warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Define regex patterns based on the allowed units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units set based on entity_unit_map
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Preprocessing function to clean up OCR text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    
    # Replace "O" with "0" (to handle common OCR errors like "140OmG")
    text = text.replace("o", "0")
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove any unnecessary punctuation that might interfere with matching
    text = re.sub(r'[^\w\s.,]', '', text)
    
    return text

# Function to extract various units from the text
def extract_units(text, entity_name):
    extracted_units = set()

    # Preprocess the text before extracting units
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        unit_list = entity_unit_map[entity_name]
        for unit in unit_list:
            if unit in patterns:
                pattern = patterns[unit]
                matches = re.findall(pattern, cleaned_text)
                if matches:
                    # Format as "value unit" and store unique entries
                    for match in matches:
                        value = float(match[0])
                        formatted_entry = f"{value} {unit}"
                        extracted_units.add(formatted_entry)

    return extracted_units

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0
    
    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        
        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)
        
        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        
        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle
    
    # Return the best text and angle
    return best_text, best_angle

# Function to process each image URL
def process_image_url(image_url, entity_name):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units from the final text based on the entity_name
    units = extract_units(final_text, entity_name)
    
    return final_text, best_angle, units

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Limit to first 50 links for processing
df = df.head(100)

# Initialize a list to store results
results = []

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_link'
    entity_name = row['entity_name']  # Assuming this column specifies what entity to extract (e.g., weight, volume)
    
    # Process image
    final_text, best_angle, units = process_image_url(image_url, entity_name)
    
    # Prepare the result row
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)  # Convert set to list for the predicted result
    }
    
    # Append the result to the list
    results.append(result_row)

# Print the count of processed images
print(f"Processed {len(results)} images.")

# Create a new DataFrame from results and save it as a CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)
"""

Processed 100 images.


In [21]:
pred = pd.read_csv('predicted_units.csv')

In [22]:
new_train = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [23]:
# Merging df1 and test1 on image_link, entity_name, and group_id
pred = pred.merge(new_train, on=['image_link', 'entity_name'], how='inner')

# Optionally, if you need to remove duplicates based on these columns:
pred = pred.drop_duplicates(subset=['image_link', 'entity_name', 'group_id'])

In [24]:
# Convert 'predicted' column from a list of strings to a single string without brackets and quotes
pred['predicted'] = pred['predicted'].apply(lambda x: ', '.join(sorted(set(x.strip('[]').replace("'", "").split(', ')))))

In [25]:
pred = pred[['image_link', 'entity_name', 'extracted_text', 'predicted', 'group_id', 'entity_value']]

In [26]:
import pandas as pd
import re

# Function to convert values to a common unit (e.g., grams) for comparison
def convert_to_grams(value):
    if pd.isna(value):
        return None
    value = value.lower().strip()
    # Extract the numeric part and unit
    match = re.match(r"([\d.]+)\s*(\w+)", value)
    if match:
        number = float(match.group(1))
        unit = match.group(2)
        if unit in ["gram", "grams", "g"]:
            return number
        elif unit in ["milligram", "milligrams", "mg"]:
            return number / 1000
        elif unit in ["kilogram", "kilograms", "kg"]:
            return number * 1000
        elif unit in ["ounce", "ounces", "oz"]:
            return number * 28.3495
        elif unit in ["pound", "pounds", "lb"]:
            return number * 453.592
    return None

# Function to get the highest value in grams and return the original value
def get_highest_value(row):
    predicted_values = row['predicted'].split(',')
    highest_value = None
    original_value = None
    for value in predicted_values:
        value_in_grams = convert_to_grams(value)
        if value_in_grams is not None:
            if highest_value is None or value_in_grams > highest_value:
                highest_value = value_in_grams
                original_value = value.strip()
    return original_value if original_value is not None else ""

# Apply the function to create a new column 'predict'
pred['predict'] = pred.apply(get_highest_value, axis=1)

In [27]:
# Display the cleaned DataFrame
pred.head(30)

,image_link,entity_name,extracted_text,predicted,group_id,entity_value,predict
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,item_weight,PROPOS' NATUREJ INGREDIENT MENAGER MULTI-USAGE...,500.0 gram,748919,500.0 gram,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,item_volume,TLael=_ 767222 Xe RRIFIC; LEBENSMITTELECHT G D...,,916768,1.0 cup,
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,item_weight,COMPOSITION Serving Size: Tablet (0.709 g) Eac...,"0.04 gram, 0.09 gram, 0.2 gram, 0.51 gram, 0.7...",459516,0.709 gram,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,item_weight,conpositioN Serving Size: Tablet (0.709 g1 Eac...,"0.2 gram, 0.51 gram, 0.709 gram, 10.0 milligra...",459516,0.709 gram,57.0 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,item_weight,Horbaach' HIGH StRENGTH PSYLLIUM HUSK PLANTAGO...,1400.0 milligram,731432,1400 milligram,1400.0 milligram
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,item_weight,"4JEEQJOH sue1ue1aba, 3 suebaA JOJ aqens - buiu...",,731432,1400 milligram,
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,item_weight,"HOrbaach Directions: For adults, take two (2) ...",1400.0 milligram,731432,1400 milligram,1400.0 milligram
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,item_weight,VEGAN WHEAT FREE soy FREE OVATA DAIRY FREE FRE...,1400.0 milligram,731432,1400 milligram,1400.0 milligram
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,item_weight,Horbaach V 100% HIGHEST QUALITY TO HGH GMP ) H...,1400.0 milligram,731432,1400 milligram,1400.0 milligram
9,https://m.media-amazon.com/images/I/71jBLhmTNl...,item_weight,NEW LOOK SAME TRUSTEDQUALITY OLD Horbaach NEW ...,"1400.0 milligram, 400.0 milligram",731432,1400 milligram,1400.0 milligram


In [69]:
import re
from fuzzywuzzy import fuzz

# Conversion dictionary
unit_conversions = {
    'gram': {'kilogram': 0.001, 'milligram': 1000, 'microgram': 1e6, 'ounce': 0.03527396, 'pound': 0.00220462},
    'kilogram': {'gram': 1000, 'milligram': 1e6, 'microgram': 1e9, 'ounce': 35.27396, 'pound': 2.20462},
    'milligram': {'gram': 0.001, 'kilogram': 1e-6, 'microgram': 1000, 'ounce': 3.527396e-5, 'pound': 2.204623e-6},
    'ounce': {'gram': 28.3495, 'kilogram': 0.0283495, 'milligram': 28349.5, 'pound': 0.0625},
    'pound': {'gram': 453.592, 'kilogram': 0.453592, 'milligram': 453592, 'ounce': 16},
}

# Function to calculate the prediction and the matching score
def predict_presence(entity_value, predicted_value, threshold=70):
    # Calculate the fuzzy match ratio between the entity and the predicted value
    match_score = fuzz.partial_ratio(entity_value.lower(), predicted_value.lower())
    
    # Predict presence based on the match score exceeding the threshold
    presence = 1 if match_score >= threshold else 0
    
    return presence, match_score / 100  # Return as a probability (0 to 1)

# Function to parse the value and unit from a string (expects 'value unit' format)
def parse_value_and_unit(value_str):
    match = re.match(r'(\d+(\.\d+)?)(\s+)?([a-zA-Z]+)', value_str.strip())
    if match:
        value = float(match.group(1))
        unit = match.group(4).lower()
        return value, unit
    return None, None

# Function to convert predicted value to different units
def convert_units(value, unit):
    conversions = unit_conversions.get(unit, {})
    converted_values = {new_unit: value * factor for new_unit, factor in conversions.items()}
    return converted_values

# Function to get the best match from the predicted values
def get_best_prediction(row, threshold=50):
    predictions = row['predicted'].split(", ")  # Split multiple predicted values
    best_prediction = None
    best_probability = 0
    match_found = False
    
    # First, check if any predicted value matches directly
    for predicted_value in predictions:
        presence, probability = predict_presence(row['entity_value'], predicted_value)
        if probability > best_probability and probability > (threshold / 100):
            best_prediction = predicted_value
            best_probability = probability
            match_found = True
    
    # If no match is found, attempt to convert the predicted values and re-check
    if not match_found:
        for predicted_value in predictions:
            value, unit = parse_value_and_unit(predicted_value)
            if value is not None and unit in unit_conversions:
                converted_values = convert_units(value, unit)
                for new_unit, new_value in converted_values.items():
                    converted_str = f"{new_value} {new_unit}"  # Recreate the string for comparison
                    presence, probability = predict_presence(row['entity_value'], converted_str)
                    if probability > best_probability and probability > (threshold / 100):
                        best_prediction = converted_str
                        best_probability = probability
    
    return best_prediction, best_probability

# Apply the function to update the DataFrame
pred[['prediction', 'probability']] = pred.apply(
    lambda row: get_best_prediction(row), axis=1, result_type='expand'
)


In [70]:
# Show the DataFrame with predictions and probabilities
pred.head(30)


,image_link,entity_name,extracted_text,predicted,group_id,entity_value,prediction,probability
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,item_weight,PROPOS' NATUREJ INGREDIENT MENAGER MULTI-USAGE...,500.0 gram,748919,500.0 gram,500.0 gram,1.00
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,item_volume,TLael=_ 767222 Xe RRIFIC; LEBENSMITTELECHT G D...,,916768,1.0 cup,NaN,0.00
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,item_weight,COMPOSITION Serving Size: Tablet (0.709 g) Eac...,"0.04 gram, 0.09 gram, 0.2 gram, 0.51 gram, 0.7...",459516,0.709 gram,0.709 gram,1.00
3,https://m.media-amazon.com/images/I/612mrlqiI4...,item_weight,conpositioN Serving Size: Tablet (0.709 g1 Eac...,"0.2 gram, 0.51 gram, 0.709 gram, 10.0 milligra...",459516,0.709 gram,0.709 gram,1.00
4,https://m.media-amazon.com/images/I/617Tl40LOX...,item_weight,Horbaach' HIGH StRENGTH PSYLLIUM HUSK PLANTAGO...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
5,https://m.media-amazon.com/images/I/61QsBSE7jg...,item_weight,"4JEEQJOH sue1ue1aba, 3 suebaA JOJ aqens - buiu...",,731432,1400 milligram,NaN,0.00
6,https://m.media-amazon.com/images/I/81xsq6vf2q...,item_weight,"HOrbaach Directions: For adults, take two (2) ...",1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
7,https://m.media-amazon.com/images/I/71DiLRHeZd...,item_weight,VEGAN WHEAT FREE soy FREE OVATA DAIRY FREE FRE...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
8,https://m.media-amazon.com/images/I/91Cma3Rzse...,item_weight,Horbaach V 100% HIGHEST QUALITY TO HGH GMP ) H...,1400.0 milligram,731432,1400 milligram,1400.0 milligram,0.86
9,https://m.media-amazon.com/images/I/71jBLhmTNl...,item_weight,NEW LOOK SAME TRUSTEDQUALITY OLD Horbaach NEW ...,"1400.0 milligram, 400.0 milligram",731432,1400 milligram,1400.0 milligram,0.86


In [73]:
pred.to_csv('pred.csv', index=False)

In [10]:
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import urllib
import re
import warnings
import os
import multiprocessing
from functools import partial
from tqdm import tqdm
from pathlib import Path
import time

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Regex patterns for units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    # Other patterns...
}

# Entity-to-unit mapping
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'milligram', 'ounce', 'pound'},
    # Other entity mappings...
}

# Allowed units
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Download an image from URL with retries and placeholder on failure
def download_image(image_link, save_folder, retries=3, delay=3):
    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    # Skip if image already exists
    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception:
            time.sleep(delay)

    # Create a placeholder image in case of failure
    Image.new('RGB', (100, 100), color='black').save(image_save_path)

# Parallel image downloading function
def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    download_image_partial = partial(download_image, save_folder=download_folder)
    with multiprocessing.Pool(64) as pool:
        list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
        pool.close()
        pool.join()

# Preprocess text (lowercase, handle common OCR errors)
def preprocess_text(text):
    text = text.lower().replace("o", "0")
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,]', '', text)
    return text

# Extract units from text based on entity type
def extract_units(text, entity_name):
    extracted_units = set()
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        for unit in entity_unit_map[entity_name]:
            if unit in patterns:
                matches = re.findall(patterns[unit], cleaned_text)
                for match in matches:
                    value = float(match[0])
                    extracted_units.add(f"{value} {unit}")
    return extracted_units

# Perform OCR and extract best text by rotating image
def rotate_and_extract(image):
    best_text, best_angle = "", 0
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        rotated_img_np = np.array(rotated_img)
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        if len(final_text.strip()) > len(best_text.strip()):
            best_text, best_angle = final_text, angle
    return best_text, best_angle

# Process image URL
def process_image_url(image_url, entity_name, download_folder):
    # Download and open image
    filename = Path(image_url).name
    image_path = os.path.join(download_folder, filename)
    image = Image.open(image_path)
    
    # Extract text using OCR and find best rotation
    final_text, best_angle = rotate_and_extract(image)
    
    # Extract units
    units = extract_units(final_text, entity_name)
    return final_text, best_angle, units

# Load CSV file (replace with your path)
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path).head(10)  # Process first 100 links

# Image folder to save the downloaded images
download_folder = "downloaded_images"

# Download images in parallel
print("Starting image download...")
download_images(df['image_link'], download_folder)

# Initialize list to store results
results = []

# Start tracking time
start_time = time.time()

# Process each image and extract relevant data
for index, row in df.iterrows():
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Process image to get text, angle, and units
    final_text, best_angle, units = process_image_url(image_url, entity_name, download_folder)

    # Add results
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)
    }
    results.append(result_row)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    remaining_images = len(df) - (index + 1)
    estimated_total_time = (elapsed_time / (index + 1)) * len(df)
    remaining_time = estimated_total_time - elapsed_time

    # Print countdown
    print(f"Processed {index + 1}/{len(df)} images. Estimated time remaining: {remaining_time // 60:.0f}m {remaining_time % 60:.0f}s")

# Create DataFrame and save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)

print(f"Processed {len(results)} images.")


Starting image download...


100%|██████████| 10/10 [00:00<00:00, 4636.13it/s]


Processed 1/10 images. Estimated time remaining: 0m 48s


KeyboardInterrupt: 